In [14]:
from sklearn.model_selection import train_test_split

In [5]:
x_set = []
y_set = []

with open("dialog_acts.dat", 'r') as file:
    for line in file:
        y_set.append(line.split()[0])
        x_set.append(" ".join(line.split()[1:]))

y_set[0], x_set[0]
    

('inform', 'im looking for a moderately priced restaurant that serves')

In [16]:
# x_train, x_test = x_set[int(len(x_set)*0.15):], x_set[:int(len(x_set)*0.15)]
# y_train, y_test = y_set[int(len(y_set)*0.15):], y_set[:int(len(y_set)*0.15)]
# len(x_train), len(x_test), len(y_train), len(y_test)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(x_set, y_set, test_size=0.15, random_state=42)
len(X_train), len(X_test), len(y_train), len(y_test)


(21675, 3826, 21675, 3826)

## Dummy baseline

In [18]:
def dummy_baseline(query: str) -> str:
    return max(set(y_train), key=y_train.count)

In [19]:
print(dummy_baseline('hi'))

inform


In [24]:
dummy_baseline_acc = y_test.count('inform') / len(y_test)
print(dummy_baseline_acc)

0.40041819132253004


## Rule-based baseline : TODO

In [18]:
def rule_based_baseline(query: str, keywords_dict: dict[str, str]):
    for key, value in keywords_dict.items():
        for keyword in value:
            if keyword in query:
                return key

In [ ]:
if ['no', 'yes'] in 'no':
    print('Aha')

In [28]:
query = 'yss'
if any(keyword in query for keyword in ['yes', 'no']):
    print('aha')
    
    

In [8]:
keyword_dict = {
    'ack': ['kay', 'okay', 'good', 'fine'],
    'affirm': ['yes', 'right', 'correct', 'yeah', 'ye'],
    'bye': ['good', 'bye'],
    'confirm': ['does', 'is', 'it', 'they', 'there'],
    'deny': ['wrong', 'want', 'dont'],
    'hello': ['hi', 'hello', 'im', 'looking'],
    'inform': ['any', 'food', 'care', 'expensive', 'moderate', 'cheap', 'east', 'west', 'north', 'south', 'centre', 'care', 'town', 'area'],
    'negate': ['no'],
    'null': ['unintelligible', 'noise', 'okay', 'what', 'uh', 'sil', 'laughing'],
    'repeat': ['repeat', 'back', 'go', 'again'],
    'reqalts': ['else', 'next', 'how', 'about', 'any', 'anything'],
    'reqmore': ['more'],
    'request': ['type', 'phone', 'number', 'address'],
    'restart': ['start', 'over', 'reset'],
    'thankyou': ['thank', 'you', 'good', 'bye'],
}

In [30]:
positive = 0
for x in range(len(y_set)):
    pred = rule_based_baseline(x_set[x], keyword_dict)
    if (x_set[x] == pred):
        positive = positive+1
rate = positive / len(y_set)
print('Rate: ', rate)




Rate:  0.0011764244539429827
